In [1]:
# Passo 1: Importar as bibliotecas necessárias

import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasRegressor

In [2]:
# Passo 2: Carregar os dados da planilha 

# Definir o caminho para o arquivo CSV
caminho_arquivo = 'C:/TCC/pedidos_faturamento.csv'

# Carregar o arquivo CSV em um DataFrame
df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='utf-8', header=0)

# Exibir as primeiras linhas do DataFrame para verificar se foi carregado corretamente
print(df.head())

   Código do Item Cidade de Entrega Estado de Entrega  \
0               1           FORMIGA                MG   
1               1           FORMIGA                MG   
2               3           FORMIGA                MG   
3               4           FORMIGA                MG   
4               5           FORMIGA                MG   

  Data de Implantação do Pedido Data de Entrega do Pedido  Tempo de Entrega  
0                    20/05/2020                04/06/2020                15  
1                    22/05/2020                25/05/2020                 3  
2                    22/05/2020                25/05/2020                 3  
3                    18/06/2020                06/07/2020                18  
4                    18/06/2020                06/07/2020                18  


In [3]:
target_column = 'Tempo de Entrega'

# Separar as features e o target
X = df.drop(columns=[target_column])
y = df[target_column]

# Identificar colunas categóricas e numéricas
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=[float, int]).columns

In [4]:
# Codificar variáveis categóricas, se houver
X = pd.get_dummies(X)

In [5]:
# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
# Função para criar o modelo de Rede Neural
def create_model():
    model = Sequential()
    model.add(Input(shape=(X.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))  # Camada de saída para regressão
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    return model

In [7]:
# Envolver o modelo Keras com KerasRegressor
keras_regressor = KerasRegressor(model=create_model, epochs=50, batch_size=10, verbose=0)


In [8]:
# Definir a validação cruzada com k-fold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)


In [9]:
# Definir as métricas para avaliação
scoring = {
    'mse': make_scorer(mean_squared_error),
    'mae': make_scorer(mean_absolute_error),
    'r2': make_scorer(r2_score)
}

In [10]:
# Avaliar o modelo com cross-validation
results = cross_val_score(keras_regressor, X, y, cv=kfold, scoring='neg_mean_squared_error')
mse_scores = -results
results = cross_val_score(keras_regressor, X, y, cv=kfold, scoring='neg_mean_absolute_error')
mae_scores = -results
results = cross_val_score(keras_regressor, X, y, cv=kfold, scoring='r2')
r2_scores = results

print("Neural Network - Cross-Validation Results")
print(f"Mean Squared Error: {mse_scores.mean():.2f} ± {mse_scores.std():.2f}")
print(f"Mean Absolute Error: {mae_scores.mean():.2f} ± {mae_scores.std():.2f}")
print(f"R2 Score: {r2_scores.mean():.2f} ± {r2_scores.std():.2f}")

Neural Network - Cross-Validation Results
Mean Squared Error: 43.23 ± 8.37
Mean Absolute Error: 2.02 ± 0.21
R2 Score: 0.91 ± 0.02
